In [ ]:
import pandas as pd
import numpy as np
import argparse
import glob
# import wandb
import json
import joblib
import os

from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

CALC_PATHS = '/home/nfs/jludwiczak/af2_cc/af2_multimer/calc'

In [ ]:
import sys
import os
import glob

import pandas as pd
import numpy as np

import pytest

from src.predictor import predict_oligo_state
DATA_PATH = 'tests/data/*'
# @pytest.mark.parametrize("use_pairwise", [True, False])

def test_predict_oligo_state():
    test_df = pd.DataFrame()
    test_cases = glob.glob(DATA_PATH)
    for i, test_case in enumerate(test_cases):
        df = predict_oligo_state(test_case, use_pairwise=True)
        df['test_case_index'] = i
        df = df.set_index('test_case_index')
        test_df = pd.concat([test_df, df], axis=0)
    print(test_df)
    # test_df.to_csv('tests/test_df.csv')
    # print(test_df)
    assertion_df = pd.read_csv('tests/test_df.csv').set_index('test_case_index')
    # diff = test_df.compare(assertion_df)
    # print(diff)
    assert np.isclose(test_df, assertion_df).all(), 'Test case results do not match assertion results'

test_predict_oligo_state()

In [ ]:
import pandas as pd
df = pd.read_pickle('/home/nfs/rmadaj/pdb_scan_nr_topology_20200717.p')

In [ ]:
import sys
sys.path.append('/home/nfs/sdunin/scr/localpdb/')
from localpdb import PDB
from Bio.Data.IUPACData import protein_letters_3to1
import gzip

In [55]:
tdf = df.drop_duplicates(subset=['pdb']).reset_index(drop=True)
pdbs = tdf.head(500)[['pdb']].values.flatten()

lpdb = PDB('/home/db/localpdb/', version=20210716)


/home/users/rmadaj/anaconda3/envs/util/lib/python3.9/site-packages/localpdb/utils/prot.py:60 - Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.


In [56]:
from pdbfixer import PDBFixer
from openmm.app import PDBFile
from biopandas.pdb import PandasPdb
import shutil, os

input_dict = {}

for pdb in pdbs:
    entry = lpdb.entries.loc[lpdb.entries.index==pdb].pdb_fn.values[0]
    seq = ''.join(PandasPdb().read_pdb(entry).amino3to1()['residue_name'])
    if len(seq) > 250:
        continue
    if '?' in seq:
        continue
    input_dict[pdb] = seq





In [63]:
len(input_dict)

75

In [72]:
i = 0
for key, value in input_dict.items():
    print(key, value)
    os.makedirs(f'tmp/{i}', exist_ok=True)
    seq = value
    pdb = key
    
    with open(f'tmp/{i}/{key}_monomer.csv', 'w') as f:
        f.write('id,sequence\n')
        f.write(f'{pdb},{seq}\n')
        
    with open(f'tmp/{i}.sh', 'w') as f:
        f.write('#!/bin/bash\n')
        f.write(f'cd ./{i}\n')
        f.write('source /opt/miniconda3/bin/activate cf_1.5\n')
        f.write(f'colabfold_batch {key}_monomer.csv . --num-models 5 --model-type alphafold2_multimer_v3 --num-recycle 5 --save-single-representations --save-pair-representations\n')
    i += 1
        

6n63 KEELDAFHQIFTTTKEAIERFMAMLTPVIENAEDDHERLYYHHIYEEEEQRLSRLDVLIPLIEKFQDETDEGLFSPSNNAFNRLLQELNLEKFGLHNFIEHVDLALFSFTDEERQTLLKELRKDAYEGYQYVKEKLAEINA
4n6j STDWEVERAELQARIALQGERKGQENLKKDLVRRIKLEALKQERADWEVERAELQARIALQGERKGQENLKKDLVRRIKLEALKQERA
1szt SGIVQQQNNLLRAIEAQQHLLQLTVWGIKQLQARSGGRGGWMEWDREINNYTSLIHSLIEESQNQQEK
3sz7 GPRSMAPTPESDKLKSEGNAAMARKEYSKAIDLYTQALSIAPANPIYLSNRAAAYSASGQHEKAAEDAELATVVDPKYSKAWSRLGLARFDMADYKGAKEAYEKGIEAEGNGGSDAMKRGLETTKRKIEEANRGAEPPADDVDDAAGASRG
1sz7 KMSSELFTLTYGALVTQLCKDYENDEDVNKQLDKMGFNIGVRLIEDFLARSNVGRCHDFRETADVIAKVAFKMYLGITPSITNWSPAGDEFSLILENNPLVDFVELPDNHSSLIYSNLLCGVLRGALEMVQMAVEAKFVQDTLKGDGVTEIRMRFIRRI
1szi NRLPLTEAELALIATPPEDSDMASLQQQRQEQNYFVRLGSLSERLRNHAYEHSLGKLQNARQKAQETLQQLTSVLGLMESVKQAKPEQVEARALSMFRDITQQLQSMCVALGASIQGLPSHVREQAQQARSQVNDLQATFSGIHSFQDLSAGVLAQTRERIARAREALDNTVEYVAQNTPAMWLVGPFAPGITE
4dzu SGIVQQQNNLLRAIEAQQHLLQLTVWGIKQLQARILSSGGWMEWDREINNYTSLIHSLIEESQNQQEKNEQGGSGIVQQQNNLLRAIEAQQHLLQLTVWGIKQLQARILSGGSGGWMEWDREINNYTSLIHSLIEESQNQQEKNEQELLGGGSGIVQQ

In [70]:
with open(f'tmp/batch.sh', 'w') as f:
    f.write("""#!/bin/bash
#SBATCH -p gpu
#SBATCH -n 4
#SBATCH --gres=gpu:1
#SBATCH --mem=16GB
#SBATCH -J wio
#SBATCH --array 0-74

bash $SLURM_ARRAY_TASK_ID.sh

""")

In [67]:
[print(x) for x in input_dict.keys()]

6n63
4n6j
1szt
3sz7
1sz7
1szi
4dzu
4dzk
4dzo
2pjw
4gif
3gi7
4m1p
6m1v
1ce0
1ce9
4y5o
6vg5
3vgx
2yy0
4zmk
5xqh
5waq
3waq
1was
6is6
4r61
6r6o
1r6n
6r6m
4cya
4cy9
3cyo
3o3x
1o3u
1o3x
4pxk
6px0
4pxj
5tuu
2oqq
6nb0
6nbu
4nbo
1yhn
2yhc
6f62
6f64
3ter
3te3
6bp6
4bpg
3bpv
3a7p
3a7m
1g5z
5lb7
5lb4
1lb3
3lbw
5fzs
6fzz
2fzt
4fzp
3s9g
1s94
1s9z
6s9k
1gl2
2gf4
5kzl
6kzj
4jzl
3jz9
4jzp


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]